In [1]:
import pandas as pd

In [3]:
import dask.dataframe as dd
import pandas as pd
# Ler o arquivo Parquet
df = dd.read_parquet('C:/Users/AU355EE/OneDrive - EY/Panorama Inadimplência - Projeto Academy/Database/Database BCB/Ouro/Consolidado/consolidado_2020_a_2024.parquet')

# Visualizar as primeiras linhas
print(df.head())


   data_base  uf cliente                     ocupacao cnae_secao  \
0 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
1 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
2 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
3 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
4 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   

                                           porte  \
0  PF - Acima de 20 salários mínimos               
1  PF - Acima de 20 salários mínimos               
2  PF - Acima de 20 salários mínimos               
3  PF - Acima de 20 salários mínimos               
4  PF - Acima de 20 salários mínimos               

                                 modalidade numero_de_operacoes  \
0                    PF - Cartão de crédito                 974   
1                    PF - Cartão de crédito               <= 15   
2  PF - Empréstimo com consignação em folha                 797   
3  PF - Em

In [4]:
print("Colunas antes da remoção:")
print(df.columns)

# Remover a coluna 'tipo_cliente'
df = df.drop(columns=['tipo_cliente'])

# Exibir as colunas após a remoção
print("\nColunas após a remoção:")
print(df.columns)

Colunas antes da remoção:
Index(['data_base', 'uf', 'cliente', 'ocupacao', 'cnae_secao', 'porte',
       'modalidade', 'numero_de_operacoes', 'a_vencer_ate_90_dias',
       'carteira_ativa', 'carteira_inadimplida_arrastada',
       'ativo_problematico', 'tipo_cliente'],
      dtype='object')

Colunas após a remoção:
Index(['data_base', 'uf', 'cliente', 'ocupacao', 'cnae_secao', 'porte',
       'modalidade', 'numero_de_operacoes', 'a_vencer_ate_90_dias',
       'carteira_ativa', 'carteira_inadimplida_arrastada',
       'ativo_problematico'],
      dtype='object')


In [5]:
df["numero_de_operacoes"] = (
    df["numero_de_operacoes"]
    .str.replace(r"[^\d]", "", regex=True)  # Remove tudo que não for número
    .astype(int)  # Converte para inteiro
)

In [6]:
print(df.head())

   data_base  uf cliente                     ocupacao cnae_secao  \
0 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
1 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
2 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
3 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   
4 2020-01-31  AC      PF  PF - Aposentado/pensionista          -   

                                           porte  \
0  PF - Acima de 20 salários mínimos               
1  PF - Acima de 20 salários mínimos               
2  PF - Acima de 20 salários mínimos               
3  PF - Acima de 20 salários mínimos               
4  PF - Acima de 20 salários mínimos               

                                 modalidade  numero_de_operacoes  \
0                    PF - Cartão de crédito                  974   
1                    PF - Cartão de crédito                   15   
2  PF - Empréstimo com consignação em folha                  797   
3  PF 

In [13]:
df_agg = df.groupby([
    "data_base", "uf", "cliente", "ocupacao", "cnae_secao", "porte", 
    "modalidade", "numero_de_operacoes"
]).agg({
    "a_vencer_ate_90_dias": "sum",
    "carteira_ativa": "sum",
    "carteira_inadimplida_arrastada": "sum",
    "ativo_problematico": "sum"
}).reset_index()

In [8]:
df_agg

,a_vencer_ate_90_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
npartitions=1,,,,
,float64,float64,float64,float64
,...,...,...,...


In [9]:
df_agg.count()

Dask Series Structure:
npartitions=1
a_vencer_ate_90_dias              int64
carteira_inadimplida_arrastada      ...
Dask Name: count, 8 expressions
Expr=(GroupbyAggregation(frame=Assign(frame=Drop(frame=ReadParquetFSSpec(8c5b7c3), columns=['tipo_cliente'])), arg={'a_vencer_ate_90_dias': 'sum', 'carteira_ativa': 'sum', 'carteira_inadimplida_arrastada': 'sum', 'ativo_problematico': 'sum'}, observed=False)).count()

In [10]:
print(df_agg.dtypes)

a_vencer_ate_90_dias              float64
carteira_ativa                    float64
carteira_inadimplida_arrastada    float64
ativo_problematico                float64
dtype: object


In [ ]:
# Salvar o DataFrame em um arquivo CSV
# df_agg.to_csv('df_agg.csv', index=False)  # index=False para não salvar o índice

# print("DataFrame salvo em 'df_agg.csv'")

In [ ]:
# Salvar o DataFrame em um arquivo CSV
# df_agg.to_parquet('df_agg.parquet')  # index=False para não salvar o índice

# print("DataFrame salvo em 'df_agg.parquet'")

DataFrame salvo em 'df_agg.parquet'


In [14]:
# Supondo que df_agg seja um Dask DataFrame
df_agg = df_agg.compute()  # Converte para um DataFrame do Pandas

# Agora salve como CSV
df_agg.to_csv('df_agg.csv', index=False)
